In [1]:
sheets = {
'f19_reviewer.xlsx':'1KJpIeML2f2t6UxxGFzOTOQPM0xtyUgJVVBrW3I9nyjU', # except change first Z to K
'f19_movie.xlsx'   :'1rrAUcQxNTGcwZTnUEI5kZJvDGjLE7V1LvB5Rey5rY1w', # except change first Z to r
'f19_rating.xlsx'  :'108q2iAlSiq8AQUJCOVa5uj8lkjIpBIPLIr83C41amYw'} # except change first Z to q

import requests
for sheet in sheets:
                 url = 'https://docs.google.com/spreadsheets/d/' + sheets[sheet] + '/export?format=xlsx'
                 r = requests.get(url)
                 with open(sheet,'wb') as f:
                         f.write(r.content)
                 print('Downloaded '+sheet)

Downloaded f19_reviewer.xlsx
Downloaded f19_movie.xlsx
Downloaded f19_rating.xlsx


In [4]:
import pandas as pd

In [10]:
reviewer = pd.read_excel('f19_reviewer.xlsx')
movie = pd.read_excel('f19_movie.xlsx')
rating = pd.read_excel('f19_rating.xlsx')

rating.head()

,date,rid,mid,stars
0,2019-10-02,777,100,3
1,2019-10-02,777,101,5
2,2019-10-02,777,102,4
3,2019-10-02,777,103,5
4,2019-10-02,3333,104,1


In [16]:
moviedb = rating.merge(reviewer,on='rid').merge(movie,on='mid')

In [17]:
moviedb.head()

,date,rid,mid,stars,firstname,lastname,year,title,director
0,2019-10-02,777,100,3,John,Ringland,2014.0,Birdman,Alejandro G. Iñárritu
1,2019-10-02,1437,100,4,Zoe,Herrick,2014.0,Birdman,Alejandro G. Iñárritu
2,2019-10-04,1920,100,1,Abhishek,Mishra,2014.0,Birdman,Alejandro G. Iñárritu
3,2019-10-07,235,100,4,Jacob,Almudevar,2014.0,Birdman,Alejandro G. Iñárritu
4,2019-10-02,777,101,5,John,Ringland,2004.0,The Incredibles,Brad Bird


In [18]:
moviedb

,date,rid,mid,stars,firstname,lastname,year,title,director
0,2019-10-02,777,100,3,John,Ringland,2014.0,Birdman,Alejandro G. Iñárritu
1,2019-10-02,1437,100,4,Zoe,Herrick,2014.0,Birdman,Alejandro G. Iñárritu
2,2019-10-04,1920,100,1,Abhishek,Mishra,2014.0,Birdman,Alejandro G. Iñárritu
3,2019-10-07,235,100,4,Jacob,Almudevar,2014.0,Birdman,Alejandro G. Iñárritu
4,2019-10-02,777,101,5,John,Ringland,2004.0,The Incredibles,Brad Bird
...,...,...,...,...,...,...,...,...,...
165,2019-10-09,102,221,4,Tong,Wu,2002.0,Chicago,Rob Marshall
166,2019-10-09,102,222,5,Tong,Wu,2004.0,The Chorus,Christophe Barratier
167,2019-10-09,102,223,4,Tong,Wu,2017.0,Call Me by Your Name,Luca Guadagnino
168,2019-10-09,102,224,4,Tong,Wu,2018.0,The Ballad of Buster Scruggs,Joel Coen & Ethan Coen


              director
            /          \
        movie1         movie2
     /     |     \        |
   rating1rating2rating3 rating
 
 
 
 director 
 sum(stars)             count(movies)

In [52]:
#Which directors movies have highest rating?
directors = {}
for director in moviedb['director']:
    if director not in directors:
        directors[director] = [0,0]
    else: continue


In [53]:
for index, row in moviedb.iterrows():
    directors[row['director']][0] += row['stars']
    directors[row['director']][1] += 1

In [54]:
for director in directors:
    directors[director] = directors[director][0]/directors[director][1]

In [55]:
directors = [(k,directors[k]) for k in sorted(directors, key=directors.get, reverse=True)]

In [105]:
moviedb.groupby('director')['stars'].sum().div(moviedb.groupby('director')['stars'].count())

director
Adam Elliot                         4.000000
Akiro Kurosawa                      5.000000
Alejandro G. Iñárritu               3.000000
Andres Muschietti                   4.000000
Andrew Niccol                       4.000000
Ang Lee                             5.000000
Anthony RUsso                       3.750000
Bong Joon-ho                        5.000000
Brad Bird                           4.000000
Brenda Chapman, Mark Andrews        3.500000
Chad Stahelski                      4.750000
Christophe Barratier                5.000000
Christopher McQuarrie               5.000000
Christopher Nolan                   4.700000
Clint Eastwood                      4.500000
Darren Aronofsky                    3.333333
David Ayer                          4.000000
David Fincher                       4.000000
David Leitch                        3.000000
Denis Villeneuve                    3.500000
Edgar Wright                        4.500000
Edward Yang                         5.000000
F

In [65]:
moviedb.groupby('title')['stars'].sum().div(moviedb.groupby('title')['stars'].count())

title
2012                    2.5
21 Jump Street          4.0
A Dog's Journey         4.0
A beautiful mind        5.0
A one and a two         5.0
                       ... 
Topsy Turvy             5.0
Toy Story 4             4.0
Train to Busan          5.0
Unfriended: Dark Web    5.0
Warrior                 5.0
Name: stars, Length: 105, dtype: float64

In [103]:
pd.set_option('display.max_rows', 999)

In [104]:
a = moviedb.groupby('title')['stars'].sum().div(moviedb.groupby('title')['stars'].count()).to_frame().merge(movie, on='title').drop(columns=['mid','year']).groupby('director')['stars']
a.sum()/a.count()

director
Adam Elliot                         4.000000
Akiro Kurosawa                      5.000000
Alejandro G. Iñárritu               3.000000
Andres Muschietti                   4.000000
Andrew Niccol                       4.000000
Ang Lee                             5.000000
Anthony RUsso                       3.750000
Bong Joon-ho                        5.000000
Brad Bird                           4.000000
Brenda Chapman, Mark Andrews        3.500000
Chad Stahelski                      4.750000
Christophe Barratier                5.000000
Christopher McQuarrie               5.000000
Christopher Nolan                   4.666667
Clint Eastwood                      4.500000
Darren Aronofsky                    3.333333
David Ayer                          4.000000
David Fincher                       4.000000
David Leitch                        3.000000
Denis Villeneuve                    3.500000
Edgar Wright                        4.500000
Edward Yang                         5.000000
F